In [30]:
from cassandra.cluster import Cluster
# Connect to a local Cassandra server
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [31]:
# Test local connection
try:
    session.execute("""select * from music_library""")
except Exception as e:
    print(e) # fails due to no keyspace created

Error from server: code=2200 [Invalid query] message="No keyspace has been specified. USE a keyspace, or explicitly specify keyspace.tablename"


In [32]:
# Create a new keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e) # fails due to no keyspace created

In [33]:
# Connect to keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [34]:
query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(year int, artist_name text, album_name text, PRIMARY KEY(year, artist_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [35]:
# Check to see if the table was created; do not do this in production
query = "select count(*) from music_library"
try:
    count = session.execute(query)
except Exception as e:
    print(e)
print(count.one())

Row(count=0)


In [36]:
# Insert two rows using a template
# Cassandra does not allow for duplicates
query = "INSERT INTO music_library (year, artist_name, album_name)"
query = query + "VALUES (%s, %s, %s)"

try:
    session.execute(query, (1970,"The Beatles", "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965,"The Beatles", "Rubber Soul"))
except Exception as e:
    print(e)

In [37]:
# Validate that the data was inserted into the table; do not do this in production
query = "SELECT * FROM music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.year, row.album_name, row.artist_name)

1965 Rubber Soul The Beatles
1970 Let it Be The Beatles


In [38]:
# Make a query based on partition key - year; do not do this in production
query = "SELECT * FROM music_library WHERE YEAR=1970"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.year, row.album_name, row.artist_name)

1970 Let it Be The Beatles


In [39]:
# Delete the whole table
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [40]:
session.shutdown()
cluster.shutdown()